In [1]:
import numpy as np

In [2]:
with open('Gowalla.txt', 'r') as f:
    data = f.readlines()
    
nodes = set() # users
edges = [] # user links
similarities = [] 

for entry in data:
    node_1, node_2 = entry.split()
    
    nodes.add((int(node_1)))
    edges.append((int(node_1), int(node_2)))

    similarities.append(1) # 1 for similarity

data = None
    
for node in nodes:
    edges.append((node, node)) #self 

similarities = np.array(similarities, dtype=np.int8)


In [3]:
NODES_NUM = 196591
EDGES_NUM = 950327

SELF_SIMILARITY_PARAMETER = -2 # custom self-similarity parameter, can be changed
TRANSITIVITY = 5
ITERATIONS = 10

TOP_NUMBER = 10 # required to form top-10


SIMILARITIES_NUM = len(similarities)
print(SIMILARITIES_NUM)


1900654


In [4]:
def get_responsibility(S, A, edges):
    
    V = S + A 
    R = S.copy()
    
    # two max is required to avoid self value being max value
    max_sum_1 = np.repeat(-np.inf, NODES_NUM)
    max_sum_2 = np.repeat(-np.inf, NODES_NUM)
    
    # to store node 2 for which node 1 has max V
    paired_max = np.repeat(np.inf, NODES_NUM) 


    for i, (node_1, node_2) in enumerate(edges):
        # two max V for each node
        if max_sum_1[node_1] < V[i]:
            max_sum_2[node_1] = max_sum_1[node_1]
            max_sum_1[node_1] = V[i]

            paired_max[node_1] = node_2

        elif max_sum_2[node_1] < V[i]:
            max_sum_2[node_1] = V[i]
            
    # update R
    for i, (node_1, node_2) in enumerate(edges):
        if paired_max[node_1] == node_2:
            R[i] = R[i] - max_sum_2[node_1]
        else:
            R[i] = R[i] - max_sum_1[node_1]
    
    return R

In [5]:
def get_availability(R, A, edges):
    
    R_copy = R.copy()
    A_copy = A.copy()
   
    # max (0, R-value)
    # if max (0, R-value) < 0 => max (0, R-value) = 0 (if negative make it zero)
    
    i = 0
    while i < EDGES_NUM * 2:
        if R_copy[i] < 0:
            R_copy[i] = 0
        i += 1
            
    R_sum = np.repeat(0, NODES_NUM) #np.full(0,n)
    
    for i, (node_1, node_2) in enumerate(edges):
        R_sum[node_2] += R_copy[i]
    
    # update A
    for i, (node_1, node_2) in enumerate(edges):
        A[i] = R_sum[node_2] - R_copy[i]
        
    # find min
    i = 0
    while i < EDGES_NUM * 2:
        A[i] = np.min([0, A[i]])
        i += 1
    
    return A


In [6]:
def get_leaders(A, R, edges, transitivity):
    
    AR_sum = A + R
    
    leaders = dict()
    max_tmp = np.repeat(-np.inf, NODES_NUM)
    

    # for every node 1 find node 2 with max AR_sum
    for i, (node_1, node_2) in enumerate(edges):
        if AR_sum[i] > max_tmp[node_1]:   # update max if AR_sum[i] > saved value
            max_tmp[node_1] = AR_sum[i]
            leaders[node_1] = node_2
    
    for i in range(transitivity):        
        for node in leaders.keys(): 
            leader_value = leaders[node]
            if leaders[leader_value] != leader_value: # self
                new_leader = leaders[leader_value]
                leaders[node] = new_leader # update leader for node
                
    return leaders

In [7]:
def affinity_propagation(s, edges, self_similarity, iterations):

    S = np.append(s, np.repeat(self_similarity, NODES_NUM)) # similarity + self-similarity
    
    R = np.repeat(0, S.shape[0])
    A = np.repeat(0, S.shape[0])

    for i in range(iterations):
        R = get_responsibility(S, A, edges)
        A = get_availability(R, A, edges)
        print("Iteration №", i + 1, "in progress...")
        
    result = get_leaders(R, A, edges, TRANSITIVITY)
    
    return result

In [8]:
clusters_leaders = affinity_propagation(similarities, edges, SELF_SIMILARITY_PARAMETER, ITERATIONS)

Iteration № 0 in progress...
Iteration № 1 in progress...
Iteration № 2 in progress...
Iteration № 3 in progress...
Iteration № 4 in progress...
Iteration № 5 in progress...
Iteration № 6 in progress...
Iteration № 7 in progress...
Iteration № 8 in progress...
Iteration № 9 in progress...


In [9]:
clusters = sorted(list(set(clusters_leaders.values())))

print("Total number of clusters:", len(clusters)) # unique values 


Total number of clusters: 24801


In [10]:
# ------------- PART 2 ----------------

In [11]:
import pandas as pd

In [12]:
clusters_df=(list(clusters_leaders.values()))

print(len(clusters_df))
df_clusters = pd.DataFrame(clusters_df)
df_clusters = df_clusters.reset_index()
df_clusters.columns = ["user_id", "cluster_id"]
df_clusters.head()

196591


,user_id,cluster_id
0,0,0
1,1,1
2,2,2
3,3,3
4,4,4


In [13]:
data = None

with open('Gowalla_totalCheckins.txt', 'r') as f:
    data = f.readlines()

checkins = []
for string in data:
    location = int(string.split()[-1])
    user = int(string.split()[0])
    checkins.append((user, location)) # collect checkins per user
    
    
user_with_checkins = set()
for checkin in checkins:
    user_with_checkins.add(checkin[0])
    
print("Total number of users with checkins: ", len(user_with_checkins))

Total number of users with checkins:  107092


In [14]:
# split into test and train 
# test sample size is 7092
test = set()
train = set()
for checkin in checkins:
    user = checkin[0]
    if len(test) <= 7092:
        test.add(user)
    elif user not in test:
        train.add(user)

In [15]:
locations = dict()
for (user, location) in checkins:
    if user in train:
        if location in locations.keys():
            locations[location] += 1
        else:
            locations[location] = 1
            
            
# sort by checkins and get top-10 locations
sorted_locations = sorted(locations.items(), key = lambda loc: loc[1], reverse=True)

locations = [[key, value] for key, value in sorted_locations]


top_10 = []
for location in locations[:TOP_NUMBER]:
    top_10.append(location[0])
    
print("Top-10 locations: ", top_10)    

Top-10 locations:  [55033, 58725, 19542, 66171, 10259, 9410, 14470, 22831, 10190, 23256]


In [48]:
# metric
hits = 0
for (user, location) in checkins: 
    if user in test:
        if location in top_10:  # how many test users checked in in certain loction
            hits += 1
            
metric_1 = (hits / TOP_NUMBER) / len(test)
metric_1

0.15388411109544622

In [49]:
cluster_locations = dict()

for (user, location) in checkins:
    if user in train:
        leader = clusters_leaders[user]
        if leader in cluster_locations.keys():
            if location in cluster_locations[leader].keys():
                cluster_locations[leader][location] += 1     # number of checkins for each location each cluster (leader)
            else:
                cluster_locations[leader][location] = 1
        else:
            cluster_locations[leader] = dict()

In [50]:
# sort locations in descending 
for leader in cluster_locations:
    sorted_cluster_locations = sorted(cluster_locations[leader].items(), key = lambda loc: loc[1], reverse=True)
    cluster_locations[leader] = [[key, value] for key, value in sorted_cluster_locations]
    
        
top_10_per_cluster = dict() 

    
for leader in cluster_locations:
    for location in cluster_locations[leader][:TOP_NUMBER]:
        if leader not in top_10_per_cluster:
            top_10_per_cluster[leader] = []
        top_10_per_cluster[leader].append(location[0])
    
print(top_10_per_cluster[0])   


[33793, 188705, 27488, 31259, 608698, 398421, 19542, 280538, 74677, 9410]


In [51]:
# metric for clusters
hits = 0
for (user, location) in checkins:
    if user in test:
        leader = clusters_leaders[user]
        if leader in top_10_per_cluster and location in top_10_per_cluster[leader]:
            hits += 1
            
metric_2 = (hits / TOP_NUMBER) / len(test)
metric_2

0.2764415621034823

In [52]:
print("Precision: ", metric_1)
print("Precision for clusters: ", metric_2)

Precision:  0.15388411109544622
Precision for clusters:  0.2764415621034823
